In [1]:
from SimuSamp.new_funcs.load.SpatDat import SpatDat
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from SimuSamp.new_funcs.compute.n_core_sampler import sample_n_cores
import pingouin as pg

In [2]:
parent_filepath = "C:/Users/Michael/Desktop/block_data"
sampleids = ["01_F", "01_G", "02_G", "02_H", "03_B", "03_H", "04_I", "04_N", "05_G", "05_I", "06_H", "07_K", "08_J", "09_E", "10_B", "11_E", "11_F", "12_M", "13_G", "14_I", "15_G", "16_B", "17_I", "18_E", "19_F", "21_I", "22_N", "23_B", "24_I", "25_M", "26_I", "27_H", "28_FSC", "29_L", "30_D"]
microns_per_pixel=0.22715
mm2_per_pixels2 = (microns_per_pixel / 1000) ** 2

In [ ]:
# single iteration
simdata_list = []
for sampleid in sampleids:
    
    spatdat = SpatDat(sampleid=sampleid, parent_filepath=parent_filepath)
    
    cells = spatdat.subset_cells("extended_partition")
    tum = spatdat.subset_annotation("tumour")
    ext_partition = spatdat.subset_annotation("extended_partition")
    im = spatdat.subset_annotation("IM")
    outer_im = spatdat.subset_annotation("outer_IM")

    for region in ["tumour"]:
        for r in [0.5]:
            for n in [3]:
                for i in range(1):
                    simdata = sample_n_cores(sampleid, cells, tum, region=region, secondary_anno=im, outer_im_anno=outer_im, core_radius=r, n_cores=n, extended_partition=ext_partition, n_neighbours=True)

                    simdata_list.append(simdata)

sim_data = pd.DataFrame(simdata_list)

In [6]:
sim_data

,Density_n_mean,Den_stdev,Den_sterr,Region,Sampleid,Radius,n_cores,Cores_actually_sampled,Counts_n_mean,Areas_n_mean,Nearest_neighbour_mean,Hopkins_mean,Density_top_core,Density_bottom_core
0,679.728053,439.418629,253.698464,tumour,01_F,500.0,3,3,533.000000,0.784137,21.746482,0.768228,1180.915905,110.949982
1,166.626976,193.565414,111.755044,tumour,01_G,500.0,3,3,123.666667,0.656520,30.361446,0.641554,438.030715,0.000000
2,1111.905286,308.956212,178.375952,tumour,02_G,500.0,3,3,840.333333,0.743687,14.973381,0.821361,1523.968150,780.040400
3,497.391605,113.415830,65.480660,tumour,02_H,500.0,3,3,374.000000,0.752810,20.357022,0.813002,628.716567,351.979255
4,111.437170,133.987754,77.357866,tumour,03_B,500.0,3,3,78.000000,0.752851,44.050366,0.733787,299.878756,0.000000
5,122.649921,72.110554,41.633048,tumour,03_H,500.0,3,3,92.666667,0.751959,34.099662,0.732562,224.614402,70.140794
6,290.079646,68.821102,39.733882,tumour,04_I,500.0,3,3,222.000000,0.763181,20.607713,0.747416,385.136721,224.450539
7,614.725303,139.064482,80.288916,tumour,04_N,500.0,3,3,428.000000,0.709283,15.320555,0.818215,721.886404,418.330997
8,620.731202,393.002626,226.900172,tumour,05_G,500.0,3,3,481.000000,0.734807,22.081825,0.771623,1075.067071,116.325492
9,880.342826,53.859644,31.095880,tumour,05_I,500.0,3,3,585.000000,0.662195,16.071105,0.815010,956.214985,836.588373


#### Intraclass Correlation Coefficient

In [5]:
samples = []
tum_dens = []
im_dens = []
for sampleid in sampleids:
    spatdat = SpatDat(sampleid, parent_filepath)
    tum_den = (len(spatdat.subset_cells("tumour")) / (spatdat.subset_annotation("tumour").area * mm2_per_pixels2))
    im_den = (len(spatdat.subset_cells("IM")) / (spatdat.subset_annotation("IM").area * mm2_per_pixels2))
    samples.append(sampleid)
    tum_dens.append(tum_den)
    im_dens.append(im_den)

den = pd.DataFrame({"Sampleid": samples, "tum_den": tum_dens, "im_den": im_dens})

In [6]:
den

,Sampleid,tum_den,im_den
0,01_F,804.022884,815.782184
1,01_G,225.027071,212.653527
2,02_G,1049.946522,941.210820
3,02_H,431.779689,416.352022
4,03_B,33.618035,162.369037
5,03_H,91.502656,321.243087
6,04_I,346.394474,480.472781
7,04_N,363.616220,546.144162
8,05_G,379.291157,802.300236
9,05_I,423.454086,847.909409


In [24]:
tma = sim_data[["Sampleid", "Region", "Density_n_mean"]]

tma_im = tma[tma["Region"] == "IM"].reset_index(drop=True)
tma_tum = tma[tma["Region"] == "tumour"].reset_index(drop=True)

# den = den.rename(columns={"sampleid": "Sampleid"})

icc_table_im = tma_im.merge(den, on="Sampleid", how="left")
icc_table_tum = tma_tum.merge(den, on="Sampleid", how="left")

icc_table_im = icc_table_im[~icc_table_im["Density_n_mean"].isna()]
icc_table_tum = icc_table_tum[~icc_table_tum["Density_n_mean"].isna()]

In [25]:
icc_table_tum_long = icc_table_tum.melt(
    id_vars="Sampleid",
    value_vars=["Density_n_mean", "tum_den"],
    var_name="raters",
    value_name="ratings"
)

icc_table_im_long = icc_table_im.melt(
    id_vars="Sampleid",
    value_vars=["Density_n_mean", "tum_den"],
    var_name="raters",
    value_name="ratings"
)

In [26]:
icc_table_tum_long

,Sampleid,raters,ratings
0,01_F,Density_n_mean,587.968742
1,01_G,Density_n_mean,375.402162
2,02_G,Density_n_mean,1202.297157
3,02_H,Density_n_mean,230.826975
4,03_B,Density_n_mean,115.207468
...,...,...,...
65,26_I,tum_den,265.230197
66,27_H,tum_den,706.907380
67,28_FSC,tum_den,127.185324
68,29_L,tum_den,116.881187


In [29]:
icc_res_tum = pg.intraclass_corr(icc_table_tum_long, targets="Sampleid", ratings="ratings", raters="raters")
icc_res_tum

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.744210,6.818916,34,35,7.095328e-08,"[0.55, 0.86]"
1,ICC2,Single random raters,0.744480,6.875724,34,34,8.822848e-08,"[0.55, 0.86]"
2,ICC3,Single fixed raters,0.746055,6.875724,34,34,8.822848e-08,"[0.55, 0.86]"
3,ICC1k,Average raters absolute,0.853349,6.818916,34,35,7.095328e-08,"[0.71, 0.93]"
4,ICC2k,Average random raters,0.853527,6.875724,34,34,8.822848e-08,"[0.71, 0.93]"
5,ICC3k,Average fixed raters,0.854561,6.875724,34,34,8.822848e-08,"[0.71, 0.93]"


In [30]:
icc_res_im = pg.intraclass_corr(icc_table_im_long, targets="Sampleid", ratings="ratings", raters="raters")
icc_res_im

,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ICC1,Single raters absolute,0.494782,2.958684,30,31,0.001791,"[0.18, 0.72]"
1,ICC2,Single random raters,0.517955,3.654076,30,30,0.000328,"[0.19, 0.74]"
2,ICC3,Single fixed raters,0.570269,3.654076,30,30,0.000328,"[0.28, 0.77]"
3,ICC1k,Average raters absolute,0.662012,2.958684,30,31,0.001791,"[0.3, 0.84]"
4,ICC2k,Average random raters,0.682438,3.654076,30,30,0.000328,"[0.32, 0.85]"
5,ICC3k,Average fixed raters,0.726333,3.654076,30,30,0.000328,"[0.43, 0.87]"


### Single Sample Test

In [3]:
spatdat = SpatDat(sampleid="13_G", parent_filepath=parent_filepath)

In [4]:
cells = spatdat.subset_cells("extended_partition")
tum = spatdat.subset_annotation("tumour")
ext_partition = spatdat.subset_annotation("extended_partition")
im = spatdat.subset_annotation("IM")
outer_im = spatdat.subset_annotation("outer_IM")

In [6]:
sample_n_cores("13_G", cells, tum, region="IM", secondary_anno=im, outer_im_anno=outer_im, core_radius=0.5, n_cores=10, extended_partition=ext_partition)

{'Density_n_mean': 212.76433846359083,
 'Den_stdev': 119.06050144587343,
 'Den_sterr': 39.686833815291145,
 'Region': 'IM',
 'Sampleid': '13_G',
 'Radius': 2201.188641866608,
 'n_cores': 10,
 'Cores_actually_sampled': 9,
 'Counts_n_mean': 144.77777777777777,
 'Areas_n_mean': 0.6989862936045439,
 'Density_top_core': 405.6898316475258,
 'Density_bottom_core': 24.23045593877351}